In [1]:
import math
filename = "out_05.txt"

In [2]:
from storage_calculations import testing, calculate_entropy, total_storage

In [3]:
testing()

hello this is a test


In [4]:
with open(filename, 'rb') as f:
    lines = f.readlines()

In [5]:
for line in lines:
    print(str(line))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [6]:
#calculate the number of bits from all the edges
print(lines[1])
print(lines[1].split()[-2])
edges = int(lines[1].split()[-2])
print(edges)
bits_per_edge = 3
print("{0} bits from edges".format(bits_per_edge * edges))

b'Lexicon has 9704836 nodes and 12667321 edges\n'
b'12667321'
12667321
38001963 bits from edges


In [7]:
#format into hist
hist = {}
for line in lines:
    if ':' in str(line): #makes sure it is a line with data
        line = str(line[:-1])
        if " : " not in line:
            continue
        symbol, occ = line.split(" : ")
        hist[str(symbol[2:])] = int(occ[:-1])

In [8]:
print(len(hist.keys()))

43


In [9]:
calculate_entropy(hist)

4.608771759591944

In [10]:
total_storage(hist, edges)
175142993.88345796
213144336

0
175142373.88345796


213144336

In [20]:
print(min(hist.values()))
print(max(hist.values()))
max_cutoff = max(hist.values())
fifths = int(max_cutoff / 5)

1
1122431


In [21]:
#calculate entropy of the set I want to use
frequency_cutoff = 3
below_cutoff = 0
label_num = 0
total_counts = 0
total_bits_below_cutoff = 0
frequencies = []
for symbol, occ in hist.items():
    if occ >= frequency_cutoff:
        print(symbol, occ)
        label_num += 1
        frequencies.append(occ)
        total_counts += occ
    else:
        below_cutoff += 1
        total_bits_below_cutoff += 7 * (len(symbol) + 1)
for i in range(len(frequencies)):
    frequencies[i] /= total_counts
print(frequencies)
entropy = 0
for f in frequencies:
    entropy -= f * math.log(f, 2)
print(entropy)
print("the entropy is {0} with {1} edges, taking {2} bits".format(entropy, edges, bits_per_edge * edges * entropy))
#this willl need to be changed in the future, assumes each label is 1 character long
print("there are {0} labels with less than {1} occurences, which use {2} bits".format(below_cutoff, frequency_cutoff, total_bits_below_cutoff))
total_bits = edges * bits_per_edge + bits_per_edge * edges * entropy + total_bits_below_cutoff
print("{0} total bits with a {1} frequency cutoff".format(int(total_bits), frequency_cutoff))

_ 23
9 28328
7 29548
8 30228
3 36315
l 600142
v 177825
z 73259
x 57699
s 706046
b 281889
a 1084057
t 725622
e 1122431
n 731012
1 45216
. 443986
c 522702
m 402728
r 798671
i 862939
f 217566
* 5041
y 204716
p 335013
g 296687
- 214113
d 422375
k 201641
o 811708
5 31348
h 335431
u 385832
j 69276
2 46342
4 37532
w 193636
q 30757
6 29681
0 37955
[1.8156963953532066e-06, 0.002236306412502854, 0.002332617264778111, 0.0023862987234233362, 0.002866826721619639, 0.04737720287391583, 0.014038096152334086, 0.0057833087924861115, 0.004554950709368899, 0.05573761639798044, 0.022253253964770438, 0.08557906031553962, 0.05728301086039063, 0.08860843133620414, 0.05770851536347558, 0.00356950122662133, 0.03504972955596908, 0.04126383205408312, 0.031792685995991575, 0.06304974155535395, 0.06812327094390003, 0.017175382693539815, 0.00039795328386850064, 0.01616096101178813, 0.026447038978107122, 0.023421457236876383, 0.016902791404272224, 0.0333436854342309, 0.015918210298061564, 0.06407892563823307, 0.0024

In [22]:
#param sweep
for frequency_cutoff in range(6):
    #calculate entropy of the set I want to use
    below_cutoff = 0
    label_num = 0
    total_counts = 0
    total_bits_below_cutoff = 0
    frequencies = []
    for symbol, occ in hist.items():
        if occ >= frequency_cutoff * fifths:
#             print(symbol, occ)
            label_num += 1
            frequencies.append(occ)
            total_counts += occ
        else:
            below_cutoff += 1
            total_bits_below_cutoff += 7 * (len(symbol) + 1)
    for i in range(len(frequencies)):
        frequencies[i] /= total_counts
#     print(frequencies)
    entropy = 0
    for f in frequencies:
        entropy -= f * math.log(f, 2)
#     print(entropy)
    # entropy = label_num / total_counts #redo this!!!
#     print("the entropy is {0} with {1} edges, taking {2} bits".format(entropy, edges, bits_per_edge * edges * entropy))
    #this willl need to be changed in the future, assumes each label is 1 character long
#     print("there are {0} labels with less than {1} occurences, which use {2} bits".format(below_cutoff, frequency_cutoff, total_bits_below_cutoff))
    total_bits = edges * bits_per_edge + bits_per_edge * edges * entropy + total_bits_below_cutoff #why do we add edges * bits_per_edge twice?
    print("{0} total bits with a {1} frequency cutoff".format(int(total_bits), frequency_cutoff * fifths))

213144336 total bits with a 0 frequency cutoff
191613557 total bits with a 224486 frequency cutoff
162865626 total bits with a 448972 frequency cutoff
151183598 total bits with a 673458 frequency cutoff
75996208 total bits with a 897944 frequency cutoff
38002551 total bits with a 1122430 frequency cutoff


In [23]:
total_bits = edges * bits_per_edge + bits_per_edge * edges * entropy + 14 * below_cutoff
print(f"{total_bits} total bits from nodes+edges with a {frequency_cutoff} frequency cutoff")
#steve can you confirm this is correct?

38002551.0 total bits from nodes+edges with a 5 frequency cutoff


In [24]:
import math

#calculate codebook size
#use 2.6.2 and 2.6.3 in compact data structures book
#or just the canonical huffman wikipedia page is actually very helpful

#you can store a canonical huffman codebook as a list of symbols in order of mots frequent to least
#and then the number of symbols for each bit-length
#example: A = 11, B = 0, C = 101, D = 100
#becomes (1,1,2), ('B','A','C','D')
#so to calculate size we take the size of a list of ints (probably normal size maybe long? ask steve) of list size log base 2 of total symbols
#and add the length of a list of labels which is 7(n+1) for each label where n is label size
labels = hist.keys()
print("labels: ", list(labels))
label_bits = 0
for l in labels:
    label_bits += 7 * (len(l) + 1)
print("bits from labels: ", label_bits)
print("total labels: ", len(labels))
int_size = math.ceil(math.log(len(labels), 2)) #bit size of an unsigned int in c, goes up to 4,294,967,295, no idea if that is enough, ask steve
print(int_size)
int_size = math.ceil(math.log(int_size + 1, 2))
print(int_size)
bit_length_storage =  int_size * math.ceil(math.log(len(labels), 2))
print("codebook bit lengths storage size: ", bit_length_storage)
print("total codebook size: ", str(bit_length_storage + label_bits) + " bits")

labels:  ['/', '_', '9', '7', '8', '3', 'l', 'v', 'z', 'x', 's', 'b', 'a', ':', 't', 'e', 'n', '@', '1', '.', 'c', 'm', 'r', 'i', 'f', '*', 'y', 'p', 'g', '-', 'd', 'k', 'o', '5', 'h', 'u', 'j', '2', '4', 'w', 'q', '6', '0']
bits from labels:  602
total labels:  43
6
3
codebook bit lengths storage size:  18
total codebook size:  620 bits


In [19]:
# sweep with everything later